# `--- Day 17: Spinlock ---`

## Part 1

In [1]:
class Spinlock(object):
    def __init__(self, step=3):
        self.spinlock = [0]
        self.current_index = 0
        self.step = step

    def current_value(self):
        return self.spinlock[self.current_index]
    
    def add_one(self):
        new_index = (self.current_index + self.step) % self.len() + 1
        new_val = self.current_value() + 1
        self.spinlock.insert(new_index, new_val)
        self.current_index = new_index
        return self

    def len(self):
        return len(self.spinlock)
    
    def __str__(self):
        return ' '.join(f'({i})' if i == self.current_value() else f'{i}'
                       for i in self.spinlock)

    __repr__ = __str__

In [2]:
%%timeit -n 1 -r 1
s = Spinlock(step=370)

for i in range(2017):
    s.add_one()

print(f'part 1 answer: {s.spinlock[s.current_index + 1]}')

part 1 answer: 1244
5.22 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


## Part 2
The key to part 2 is the recurrence relation between the insertion position at step $n$: $k_n$) and the insertion postion at the previous step: $k_{n-1}$:-

$$
k_n = 1 + [k_{n-1} + 370]\pmod n
$$

with the first few positions being:

$$
k_0 = 0\\
k_1 = 1 + [370 + k_0]\pmod 1 = 1\\
k_2 = 1 + [370 + k_1]\pmod 2 = 2\\
k_3 = 1 + [370 + k_2]\pmod 3 = 1\\
k_4 = 1 + [370 + k_3]\pmod 4 = 4
$$

The 0 element always stays at the start of the buffer (there is no way that the position after the current position can be the 0 element since the current position can never be less than 0)

The only time that the value following 0 changes is when the insertion position is at 1.  We therefore step through to 50,000,000, calculating the next offset via the recurrence relation each time.  The last time this is equal to 1 prior to the 50,000,000th iteration is the value we are looking for.

In [3]:
%%timeit -n 1 -r 1
last = 0
highest = 0
for i in range(50000000):
    n = i + 1
    k_n = (last + 370) % n + 1
    if k_n == 1:
        highest = n
    last = k_n
print(f'part 2 answer: {highest}')

part 2 answer: 11162912
19.1 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
